In [ ]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from skimage.io import imread
from skimage.transform import resize
import torch.nn as nn
from torch.optim import Adam
import gc

#constants
grayscale = True
height = 256
width = 256
train = {'id':[],'label':[]}

#read input file
with open('/content/gdrive/My Drive/Colab Notebooks/ml-for-games-mid-term/pressAndpic.txt','r') as f:
    t = f.readlines()

#split data to feature-label
for i in t:
    a,b = i.split("  ")
    lb = b.replace("\n","")
    train['id'].append(a)
    train['label'].append(lb)

#preprocess the image
def preprocess(img):
    img_size = (height, width)
    img = resize(np.array(img), img_size)
    img = np.transpose(img)
    img = img.astype('float32')
    return img

train_img = []
for img_path in tqdm(train['id']):
    img = imread(img_path, as_gray=True)
    train_img.append(preprocess(img))

#defining the train data
train_x = np.array(train_img)
# defining the train label
train_y = np.array(train['label'])

#divide into train and validation set
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1)

#set number of epochs
n_epochs = 50   #44 normal and #5 is lab test
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []

# converting the target into torch format
val_x = val_x.reshape(val_x.shape[0], 1,val_x.shape[1],val_x.shape[2])
val_x  = torch.from_numpy(val_x)

val_y = val_y.astype(int)
val_y = torch.from_numpy(val_y)

x_val, y_val = val_x,val_y
y_val = y_val.type(torch.LongTensor)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.cnn_layers = nn.Sequential(
        # Defining a 2D convolution layer
        nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        # Defining another 2D convolution layer
        nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.linear_layers = nn.Sequential(
            nn.Linear(4096,1024),
            nn.Linear(1024,512),
            nn.Linear(512,2),
        )

    # Defining the forward pass
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=3e-4, weight_decay= 1e-5)
# defining the loss function
criterion = nn.CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

KeyboardInterrupt: ignored

In [ ]:
PATH = '/content/gdrive/My Drive/Colab Notebooks/ml-for-games-mid-term/val_10102020.pt'

def train(epoch, new_x_train, new_y_train):
    model.train()
    tr_loss = 0
    # converting the data into GPU format
    # clearing the Gradients of the model parameters
    optimizer.zero_grad()

    # prediction for training and validation set
    output_train = model(new_x_train)

    running_corrects_train = 0
    #computing the accuracy
    _, preds_train = torch.max(output_train, 1)
    running_corrects_train += torch.sum(preds_train == new_y_train)

    # computing the training and validation loss
    loss_train = criterion(output_train, new_y_train)
    train_losses.append(loss_train.item())

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()

    model.eval()
    output_val = model(x_val)
    running_corrects_val = 0
    _, preds_test = torch.max(output_val, 1)
    running_corrects_val += torch.sum(preds_test == y_val)
    loss_val = criterion(output_val, y_val)
    val_losses.append(loss_val.item())

    if epoch % 2 == 0:
        # printing the validation loss
        print('Epoch : ', epoch + 1, '\t', 'loss :', loss_val, 'train_acc :', running_corrects_train.double(), 'val_acc :', running_corrects_val.double())

x_val = x_val.to('cuda:0')
y_val = y_val.to('cuda:0')

min_error = float('+inf')
minima_flag = 0
best_model = Net()
tolerance = 20
# training the model
for epoch in range(n_epochs):
  val_losses = []
  for iteration in range(32):
    indices = np.random.choice(len(train_x),64)
    ipx = torch.from_numpy(train_x[indices].astype(np.float32)).to('cuda:0')
    ipy = torch.from_numpy(train_y[indices].astype(np.long)).to('cuda:0')
    ipx = ipx.view(ipx.shape[0],1,ipx.shape[1],ipx.shape[2])
    train(epoch,ipx,ipy)
    del ipx
    del ipy
    # gc.collect()
    # torch.cuda.empty_cache()
  mean_iteration_error = np.mean(val_losses)
  if mean_iteration_error < min_error:
    min_error = mean_iteration_error
    best_model = model.cpu().state_dict()
    model.cuda()
    #torch.save(model.cpu().state_dict(), PATH)
    minima_flag = 0
  else:
    minima_flag += 1
    if minima_flag > tolerance:
      break

print(str(min_error))


Epoch :  1 	 loss : tensor(0.6918, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : tensor(38., device='cuda:0', dtype=torch.float64) val_acc : tensor(158., device='cuda:0', dtype=torch.float64)
Epoch :  1 	 loss : tensor(0.6923, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : tensor(33., device='cuda:0', dtype=torch.float64) val_acc : tensor(158., device='cuda:0', dtype=torch.float64)
Epoch :  1 	 loss : tensor(0.6914, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : tensor(29., device='cuda:0', dtype=torch.float64) val_acc : tensor(158., device='cuda:0', dtype=torch.float64)
Epoch :  1 	 loss : tensor(0.6898, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : tensor(48., device='cuda:0', dtype=torch.float64) val_acc : tensor(158., device='cuda:0', dtype=torch.float64)
Epoch :  1 	 loss : tensor(0.6893, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : tensor(46., device='cuda:0', dtype=torch.float64) val_acc : tensor(158., device='cuda:0', dtype=torch

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!ls

sample_data


# **Saving the pytorch network graph**


```
# This is formatted as code
```



In [ ]:
from google.colab import files
files.download('Digraph.gv.pdf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# New Section

Saving the model parameters

Loading the model into variable

In [ ]:
model1 = Net()
model1.load_state_dict(torch.load(PATH))
model1.eval()

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=4096, out_features=1024, bias=True)
    (1): Linear(in_features=1024, out_features=

In [ ]:
pip install torchviz

     |████████████████████████████████| 51kB 3.2MB/s 
  Created wheel for torchviz: filename=torchviz-0.0.1-cp36-none-any.whl size=3520 sha256=dad7aaa8d5002334166569ea9c66dc1801df51acfbd04bfa0e68f838163bdfdf
  Stored in directory: /root/.cache/pip/wheels/2a/c2/c5/b8b4d0f7992c735f6db5bfa3c5f354cf36502037ca2b585667
Successfully built torchviz


In [ ]:
from torchviz import make_dot
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.cnn_layers = nn.Sequential(
        # Defining a 2D convolution layer
        nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        # Defining another 2D convolution layer
        nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.linear_layers = nn.Sequential(
            nn.Linear(4096,1024),
            nn.Linear(1024,512),
            nn.Linear(512,2),
        )

    # Defining the forward pass
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

model = Net()

x = torch.randn(1, 1, 256, 256).requires_grad_(True)
y = model(x)
vis_graph = make_dot(y, params=dict(list(model.named_parameters()) + [('x', x)]))
vis_graph.view()

'Digraph.gv.pdf'

THIS IS FOR LAB WORK TESTING

In [ ]:
pip install gpuinfo

In [ ]:
from gpuinfo import GPUInfo
pid_list,percent,memory,gpu_used= GPUInfo.get_info()
print(pid_list)
print(percent)
print(memory)
print(gpu_used)


{}
[0]
[1015]
[]


In [ ]:
available_device=GPUInfo.check_empty()
print(available_device)
percent,memory=GPUInfo.gpu_usage()
for i in range(len(percent)):
  print(str(percent[i]) +"  "+ str(memory[i]))

[0]
0  1015
